## Agent with Automatic Memory Persistence

Using LangGraph's MemorySaver, the agent automatically remembers conversation context without manual history management.

# LangChain Agents with Persistent Memory (Checkpointer)

This notebook demonstrates how to use LangGraph's checkpointer for automatic conversation memory persistence.

## Key Concepts:
- **MemorySaver**: Automatically saves conversation state
- **thread_id**: Identifies different conversation threads
- **Stateful Agents**: Agents that remember context across separate invocations
- **No Manual History Management**: Unlike notebook 2.1, memory is automatic

## Why Use Checkpointer?
- Conversations persist even when agent is recreated
- No need to manually track message history
- Supports multiple concurrent conversations (different thread_ids)
- Production-ready memory management

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# Create a checkpointer for automatic memory persistence
# This saves conversation state across agent invocations
checkpointer = MemorySaver()

# Define a simple tool for the agent
@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    return f"Weather in {city}: 72°F, sunny"

# Initialize language models
gptLLM = ChatOpenAI(model_name="gpt-4", temperature=0)
ollamaLLM = ChatOllama(model="llama3-groq-tool-use")

# Create agent WITH checkpointer for memory persistence
agent = create_agent(
    model=ollamaLLM,
    tools=[get_weather],
    system_prompt="You are a helpful assistant.",
    checkpointer=checkpointer  # KEY: Automatically saves state
)

# Configuration with thread_id to identify this conversation
# Different thread_ids = different conversation memories
config = {"configurable": {"thread_id": "user_123"}}

# First invocation: User introduces themselves and asks about weather
result1 = agent.invoke(
    {"messages": [HumanMessage(content="My Name is Ron. What's the weather like in New York City?")]},
    config  # Pass thread_id to save conversation state
)
print("First message response:")
print(result1["messages"][-1].content)
print("\n" + "="*80 + "\n")

# Second invocation (could be minutes or days later!)
# The agent remembers the previous conversation because:
# 1. Same thread_id is used
# 2. Checkpointer automatically loaded saved state
result2 = agent.invoke(
    {"messages": [HumanMessage(content="What is my Name?")]},
    config  # Same thread_id = retrieves saved memory
)
print("Second message response:")
print(result2["messages"][-1].content)
# Expected: Agent remembers "Ron" from the previous turn!